In [190]:
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time

In [246]:
env = gym.make("FrozenLake-v1")
env.reset()                    
print(env.step(2))
env.render()

(4, 0.0, False, {'prob': 0.3333333333333333})
  (Right)
SFFF
FHFH
FFFH
HFFG


In [249]:
env = gym.make("FrozenLake8x8-v1")
env.reset()                    
print(env.step(2))
env.render()

(0, 0.0, False, {'prob': 0.3333333333333333})
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [77]:
print("Action space: ", env.action_space)
print("Observation space: ", env.observation_space)

Action space:  Discrete(4)
Observation space:  Discrete(64)


In [76]:
env.nS

64

In [83]:
print(env.P[4][1])

[(0.3333333333333333, 3, 0.0, False), (0.3333333333333333, 12, 0.0, False), (0.3333333333333333, 5, 0.0, False)]


In [236]:
def value_iteration(env, max_iterations=100000, lmbda=0.99, theta=1e-20):
    env.reset()
    stateValue = np.zeros(env.nS)
    bestPolicy = np.zeros(env.nS)
    for i in range(max_iterations):
        newStateValue = np.copy(stateValue)
        for state in range(env.nS):
            action_values = []      
            for action in range(env.nA):
                next_state_rewards = []
                for next_sr in env.P[state][action]:
                    prob, next_state, reward, done = next_sr
                    next_state_rewards.append(prob * (reward + lmbda*newStateValue[next_state]))      #the value of each action
                action_values.append(np.sum(next_state_rewards))
            stateValue[state] = max(action_values)  #update the value of the state
            bestPolicy[state] = action_values.index(max(action_values))
        
        if (np.sum(np.fabs(newStateValue-stateValue)) <= theta):
            print("Value-iteration converged at iteration #",i)
            break
    
    return stateValue, bestPolicy

In [109]:
env.reset()  
v, p = value_iteration(env)

Value-iteration converged at iteration # 1125


In [110]:
print(v)
print(np.reshape(p,(8,8)))

[0.41464036 0.42720522 0.44614822 0.46832037 0.49244371 0.51656983
 0.53526151 0.54097522 0.41168642 0.42120783 0.43749572 0.45838855
 0.48324013 0.51353178 0.54576786 0.55736841 0.39675209 0.39384054
 0.37549627 0.         0.42167799 0.49381921 0.56121207 0.5858589
 0.36927228 0.35298254 0.30653123 0.20040371 0.30075275 0.
 0.56901589 0.62825904 0.33266395 0.29137537 0.19730918 0.
 0.28929026 0.36195181 0.53481945 0.68969732 0.30613635 0.
 0.         0.08627639 0.2139326  0.27271394 0.         0.77203552
 0.2888856  0.         0.05769641 0.04751102 0.         0.25052148
 0.         0.87776874 0.28038897 0.20081512 0.12732657 0.
 0.23959086 0.48644206 0.7371033  0.        ]
[[3. 2. 2. 2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3. 2. 2. 1.]
 [3. 3. 0. 0. 2. 3. 2. 1.]
 [3. 3. 3. 1. 0. 0. 2. 2.]
 [0. 3. 0. 0. 2. 1. 3. 2.]
 [0. 0. 0. 1. 3. 0. 0. 2.]
 [0. 0. 1. 0. 0. 0. 0. 2.]
 [0. 1. 0. 0. 1. 2. 1. 0.]]


In [247]:
for lmbda in [0.8,0.9,0.95,0.99]:
    for theta in [1e-5, 1e-10, 1e-20]:
        print(lmbda,theta)
        value_iteration(env,lmbda=lmbda,theta=theta)

0.8 1e-05
Value-iteration converged at iteration # 38
0.8 1e-10
Value-iteration converged at iteration # 79
0.8 1e-20
Value-iteration converged at iteration # 147
0.9 1e-05
Value-iteration converged at iteration # 74
0.9 1e-10
Value-iteration converged at iteration # 158
0.9 1e-20
Value-iteration converged at iteration # 266
0.95 1e-05
Value-iteration converged at iteration # 122
0.95 1e-10
Value-iteration converged at iteration # 262
0.95 1e-20
Value-iteration converged at iteration # 426
0.99 1e-05
Value-iteration converged at iteration # 288
0.99 1e-10
Value-iteration converged at iteration # 620
0.99 1e-20
Value-iteration converged at iteration # 995


In [208]:
e = 0
for i_episode in range(10000):
    c = env.reset()
    while True:
        c,reward,done,info = env.step(int(p[c]))
        #time.sleep(0.5)
        #env.render()
        if done:
            if reward == 1:
                e +=1
            break

print(e/10000)

0.8612


In [108]:
def evaluate(policy, lmbda=0.99, theta=1e-5):
    stateValue = np.zeros(env.nS)
    i = 0
    while True:
        newStateValue = np.copy(stateValue)
        for state in range(env.nS):
            action = policy[state]
            stateValue[state] = sum([prob * (reward + lmbda*newStateValue[next_state]) for prob, next_state, reward, done in env.P[state][action]])
        i += 1
        if (np.sum(np.fabs(newStateValue-stateValue)) <= theta):
            print("compute_value_function at iteration #",i)
            break
    return stateValue

In [104]:
def extract_policy(stateValue, lmbda=0.99):
    policy = np.zeros(env.nS)
    for state in range(env.nS):
        action_values = np.zeros(env.nA)
        for action in range(env.nA):
            for next_sr in env.P[state][action]:
                    prob, next_state, reward, done = next_sr
                    action_values[action] += prob * (reward + lmbda*stateValue[next_state])
        policy[state] = np.argmax(action_values)
    return policy    

In [241]:
def policy_iterations(env, max_iterations=100000, lmbda=0.99,):
    env.reset()  
    random_policy = np.zeros(env.nS)
    for i in range(max_iterations):
        value = evaluate(random_policy,lmbda)
        new_policy = extract_policy(value,lmbda)
        if (np.all(random_policy == new_policy)):
            print("policy-iteration converged at iteration #",i)
            break
        random_policy = new_policy
    return new_policy

In [163]:
p_new = policy_iterations(env)

compute_value_function at iteration # 22
compute_value_function at iteration # 382
compute_value_function at iteration # 399
compute_value_function at iteration # 409
compute_value_function at iteration # 334
compute_value_function at iteration # 340
compute_value_function at iteration # 346
compute_value_function at iteration # 357
compute_value_function at iteration # 377
compute_value_function at iteration # 381
compute_value_function at iteration # 386
policy-iteration converged at iteration # 10


In [164]:
print(p_new)
print(p)
print(p==p_new)

[3. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 2. 2. 1. 3. 3. 0. 0. 2. 3. 2. 1.
 3. 3. 3. 1. 0. 0. 2. 2. 0. 3. 0. 0. 2. 1. 3. 2. 0. 0. 0. 1. 3. 0. 0. 2.
 0. 0. 1. 0. 0. 0. 0. 2. 0. 1. 0. 0. 1. 2. 1. 0.]
[3. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 2. 2. 1. 3. 3. 0. 0. 2. 3. 2. 1.
 3. 3. 3. 1. 0. 0. 2. 2. 0. 3. 0. 0. 2. 1. 3. 2. 0. 0. 0. 1. 3. 0. 0. 2.
 0. 0. 1. 0. 0. 0. 0. 2. 0. 1. 0. 0. 1. 2. 1. 0.]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [250]:
for lmbda in [0.8,0.9,0.95,0.99]:
        print(lmbda)
        policy_iterations(env,lmbda=lmbda)

0.8
compute_value_function at iteration # 16
compute_value_function at iteration # 41
compute_value_function at iteration # 43
compute_value_function at iteration # 44
compute_value_function at iteration # 45
compute_value_function at iteration # 46
compute_value_function at iteration # 46
compute_value_function at iteration # 47
compute_value_function at iteration # 48
compute_value_function at iteration # 48
policy-iteration converged at iteration # 9
0.9
compute_value_function at iteration # 19
compute_value_function at iteration # 80
compute_value_function at iteration # 83
compute_value_function at iteration # 80
compute_value_function at iteration # 84
compute_value_function at iteration # 85
compute_value_function at iteration # 86
compute_value_function at iteration # 88
compute_value_function at iteration # 92
compute_value_function at iteration # 93
policy-iteration converged at iteration # 9
0.95
compute_value_function at iteration # 20
compute_value_function at iteration # 

In [152]:
Alpha = np.arange(0.75,1,0.02)
Lmbda = np.arange(0.10,1,0.05)
correct_train = np.zeros([len(Alpha), len(Lmbda)])
correct_test = np.zeros([len(Alpha), len(Lmbda)])

array([0.75, 0.77, 0.79, 0.81, 0.83, 0.85, 0.87, 0.89, 0.91, 0.93, 0.95,
       0.97, 0.99])

In [218]:
class QAgent(object):
    def __init__(self,obs_n,act_n,learning_rate,gamma,e_greed):
        self.obs_n = obs_n
        self.act_n = act_n
        self.lr = learning_rate
        self.gamma = gamma
        self.epsilon = e_greed
        self.Q = np.zeros([env.nS,env.nA])
    
    def predict(self,state):
        Q_list = self.Q[state,:]
        max_q = np.max(Q_list)
        action_list = np.where(Q_list==max_q)[0]
        action = np.random.choice(action_list)
        return action
        
    def sample(self,state):
        if np.random.uniform(0,1) < (1.0-self.epsilon):
            action = self.predict(state)
        else:
            action = np.random.choice(self.act_n)
        return action
    
    def learn(self,state,action,reward,next_state,done):
        predict_Q = self.Q[state,action]
        if done:
            target_Q = reward
        else:
            target_Q = reward + self.gamma * np.max(self.Q[next_state,:])
        
        self.Q[state,action] += self.lr * (target_Q - predict_Q)
              

In [219]:
class SarsaAgent(object):
    def __init__(self,obs_n,act_n,learning_rate,gamma,e_greed):
        self.obs_n = obs_n
        self.act_n = act_n
        self.lr = learning_rate
        self.gamma = gamma
        self.epsilon = e_greed
        self.Q = np.zeros([env.nS,env.nA])
    
    def predict(self,state):
        Q_list = self.Q[state,:]
        max_q = np.max(Q_list)
        action_list = np.where(Q_list==max_q)[0]
        action = np.random.choice(action_list)
        return action
        
    def sample(self,state):
        if np.random.uniform(0,1) < (1.0-self.epsilon):
            action = self.predict(state)
        else:
            action = np.random.choice(self.act_n)
        return action
    
    def learn(self,state,action,reward,next_state,next_action,done):
        predict_Q = self.Q[state,action]
        if done:
            target_Q = reward
        else:
            target_Q = reward + self.gamma * self.Q[next_state,next_action]
        self.Q[state,action] += self.lr * (target_Q - predict_Q)
              

In [220]:
def run_episode_Sarsa(env,agent,is_render=False):
    total_steps = 0
    total_reward = 0
    state = env.reset()
    action = agent.sample(state)
    while True:
        next_state, reward, done, _ = env.step(action)
        next_action = agent.sample(next_state)
        # update Q table
        agent.learn(state,action,reward,next_state,next_action,done)
        # s <- s'   a <- a'
        state = next_state
        action = next_action
        total_steps += 1
        total_reward += reward
        if is_render:
            env.render()
        if done:
            break
    return total_reward,total_steps

In [221]:
def run_episode_Q(env,agent,is_render=False):
    total_steps = 0
    total_reward = 0
    state = env.reset()
    while True:
        action = agent.sample(state)
        next_state, reward, done, _ = env.step(action)
        agent.learn(state,action,reward,next_state,done)
        state = next_state
        total_steps += 1
        total_reward += reward
        if is_render:
            env.render()
        if done:
            break
    return total_reward,total_steps

In [212]:
def test_episode(env,agent,is_render=False):
    total_reward = 0
    state = env.reset()
    
    while True:
        action = agent.predict(state)
        next_state,reward,done,_ = env.step(action)
        total_reward += reward
        state = next_state
        if is_render:
            time.sleep(0.5)
            env.render()
        if done:
            #print('test_reward', total_reward)
            break
    return total_reward

In [223]:
env.reset()
S_agent = SarsaAgent(obs_n = env.nS, act_n = env.nA, learning_rate = 0.1, gamma = 0.9, e_greed = 0.1)  
reward_his = []
for episode in range(10000):
    ep_reward, ep_steps = run_episode(env,S_agent)
    reward_his.append(ep_reward)
    #print("Episode %s: steps = %s, reward = %1.f"%(episode, ep_steps, ep_reward))
for i in range(10):
    print(sum(reward_his[i*1000:(i+1)*1000-1]))

8.0
36.0
186.0
238.0
332.0
301.0
303.0
312.0
310.0
292.0


In [224]:
env.reset()
Q_agent = QAgent(obs_n = env.nS, act_n = env.nA, learning_rate = 0.1, gamma = 0.9, e_greed = 0.1)  
reward_his = []
for episode in range(10000):
    ep_reward, ep_steps = run_episode_Q(env,Q_agent)
    reward_his.append(ep_reward)
    #print("Episode %s: steps = %s, reward = %1.f"%(episode, ep_steps, ep_reward))
for i in range(10):
    print(sum(reward_his[i*1000:(i+1)*1000-1]))

0.0
0.0
10.0
121.0
249.0
277.0
257.0
285.0
278.0
312.0


In [211]:
test_reward = 0
for i in range(1000):
    test_reward += test_episode(env,S_agent)
print(test_reward/1000)

test_reward 1.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 1.0
test_reward 1.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_rew

test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 1.0
test_reward 1.0
test_reward 1.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 1.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 1.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_reward 0.0
test_rew

In [158]:
def RL_Q(env, iterations=2000, alpha = 0.8, lmbda = 0.99):
    # alpha is learning rate; lmbda is discount factor
    # initial Q table, dim is [s,a]
    Q_all = np.zeros([env.nS,env.nA])
    rList = []
    for i in range(iterations):
        s = env.reset()
        rAll = 0
        d = False
        for i in range(1000):
            a = np.argmax(Q_all[s,:] + np.random.randn(1,env.nA) * (1./(i+1)))
            s1, r, d, _ = env.step(a)
            # update Q table
            # Q learning
            Q_all[s,a] = Q_all[s,a] + alpha*(r+lmbda*np.max(Q_all[s1,:]) - Q_all[s,a])
            rAll += r
            s = s1
            if d:
                break
        rList.append(rAll)
    
    return Q_all, sum(rList)/iterations

In [159]:
env.reset()  
Q, avg_reward = RL_Q(env)

In [160]:
avg_reward

0.099

In [ ]:
Sarsa选取的是一种保守的策略，他在更新Q值的时候已经为未来规划好了动作，对错误和死亡比较敏感。 
而Q-learning每次在更新的时候选取的是最大化Q的方向，而当下一个状态时，再重新选择动作，
Q-learning是一种鲁莽、大胆、贪婪的算法，对于死亡和错误并不在乎